In [1]:
import pandas as pd
from countryinfo import CountryInfo
import pypopulation
import plotly.graph_objects as go

In [2]:
countries = pd.read_csv('Countries.csv', delimiter=';')

In [3]:
#https://data.worldbank.org/indicator/NY.GDP.MKTP.CD
GDP = pd.read_excel('GDP_2021.xlsx')

In [4]:
def add_code(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                code = country.alt_spellings()
                code = code[0]
                return code
            except: 
                return 'Code not found'
  
        df_output = df_input.assign(code_2 = df_input.apply(temp, axis = 1))
        return df_output

In [5]:
def add_capital(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                capital = country.capital()
                return capital
            except: 
                return 'Capital not found'
  
        df_output = df_input.assign(capital = df_input.apply(temp, axis = 1))
        return df_output

In [6]:
def add_language(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                language = country.languages()
                language = list(map(lambda x: x.upper(), language))
                return language
            except: 
                return 'Language not found'
  
        df_output = df_input.assign(language = df_input.apply(temp, axis = 1))
        return df_output

In [7]:
def add_population(df_input):
        def temp(row):
            input_country =  row.country
            country = CountryInfo(input_country)
            try: 
                code =  country.alt_spellings()[0]
                population = pypopulation.get_population(code)
                population = '{:,}'.format(population).replace(',','.')
                return population
            except: 
                return 'Population not found'
  
        df_output = df_input.assign(population = df_input.apply(temp, axis = 1))
        return df_output

In [8]:
def add_GDP_2021(df_input):
        def temp(row):
            input_country =  row.country
            input_code=  row.code
            try: 
                for i in range (len(GDP)):
                    if input_country == GDP['Country Name'][i] or input_code == GDP['Country Code'][i] :
                        GDP_2021 = GDP[2021][i]
                return round(GDP_2021 / 100000000, 2)
            except: 
                return 0
  
        df_output = df_input.assign(GDP_2021_USD_in_billions = df_input.apply(temp, axis = 1))
        return df_output

In [9]:
#GDP per capita (current US$)
def add_GDP_per_capita_2021(df_input):
        def temp(row):
            try:
                GDP_2021_USD =  float(row.GDP_2021_USD_in_billions)*100000000
                population = float(row.population.replace('.',''))
                GDP_2021_per_capita = round(GDP_2021_USD / population, 1)
                return GDP_2021_per_capita
            except: 
                return 'error'
  
        df_output = df_input.assign(GDP_2021_per_capita = df_input.apply(temp, axis = 1))
        return df_output

In [10]:
countries = add_code(countries).fillna(0)
countries = add_capital(countries).fillna(0)
countries = add_language(countries).fillna(0)
countries = add_population(countries).fillna(0)
countries = add_GDP_2021(countries).fillna(0)
countries = add_GDP_per_capita_2021(countries).fillna(0)

In [11]:
countries

,country,code,code_2,capital,language,population,GDP_2021_USD_in_billions,GDP_2021_per_capita
0,Afghanistan,AFG,AF,Kabul,"[PS, UZ, TK]",38.041.754,147.87,388.7
1,Albania,ALB,AL,Tirana,[SQ],2.854.191,182.56,6396.2
2,Algeria,DZA,DZ,Algiers,[AR],43.053.054,1630.44,3787.0
3,American Samoa,ASM,AS,Pago Pago,"[EN, SM]",55.312,7.09,12818.2
4,Andorra,AND,Code not found,Capital not found,Language not found,Population not found,33.30,error
...,...,...,...,...,...,...,...,...
217,Virgin Islands,VGB,Code not found,Capital not found,Language not found,Population not found,0.00,error
218,West Bank,WBG,Code not found,Capital not found,Language not found,Population not found,0.00,error
219,Yemen,YEM,YE,Sana'a,[AR],29.161.922,0.00,0.0
220,Zambia,ZMB,ZM,Lusaka,[EN],17.861.030,221.48,1240.0


In [13]:
#https://plotly.com/python/choropleth-maps/#world-choropleth-map
import plotly.graph_objects as go

for col in countries.columns:
    countries[col] = countries[col].astype(str)

countries['text'] = 'GDP per capita ' +  countries['GDP_2021_per_capita']  + ' USD' + '<br>' + \
                    'Population ' + countries['population'] +  '<br>' + \
                    'Capital '  + countries['capital'] +  '<br>'
     

fig = go.Figure(data=go.Choropleth(
    locations = countries['code'],
    z = countries['GDP_2021_USD_in_billions'],
    text = countries['text'],
    colorscale = 'Viridis',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '$',
    colorbar_title = 'GDP<br>Billions USD',))


fig.update_layout(
    title_text='2021 Global GDP',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular',
        showlakes = False, 
        showcountries = True, 
        countrycolor="White"
    ))

fig.show()
